In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [3]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [4]:
def DataSet(iFold,dup_tag,rand_tag):
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
    #
    mat_ind_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/Mat_ind_F%d.txt") % (iFold)
    #
    mat_ind = file2array(mat_ind_path, "\t")
    
    if (int)(rand_tag):
        tmp_vec = np.zeros(num_feature)
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            tmp_vec[index] = int(mat_ind[r,c])
        index = [i for i in range(num_feature)]
        random.shuffle(index)
        tmp_vec = tmp_vec[index]
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            mat_ind[r,c] = int(tmp_vec[index])

    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, side_len, side_len, 1))
    Y_train = np.empty((nrow_train_dup, num_class))

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(side_len):
                for j in range(side_len):
                    if tmp_cnt < num_feature:
                        tmp_ind = mat_ind[i,j]
                        X_train[sam_count,i,j,0] = data_train[index,(int)(tmp_ind)]
                    else:
                        X_train[sam_count,i,j,0] = 0.5
                    tmp_cnt = tmp_cnt + 1
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1))
    Y_test  = np.empty((nrow_test, num_class))

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    tmp_ind = mat_ind[i,j]
                    X_test[index,i,j,0] = data_test[index,(int)(tmp_ind)]
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    X_train_new = np.empty((nrow_train_dup, 224, 224, 3))
    Y_train_new = Y_train
    
    for index in range(nrow_train_dup):
        tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
        for c in range(3):
            X_train_new[index,:,:,c] = tmp_new[:,:,0]
    
    X_test_new = np.empty((nrow_test, 224, 224, 3))
    Y_test_new = Y_test
    
    for index in range(nrow_test):
        tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
        for c in range(3):
            X_test_new[index,:,:,c] = tmp_new[:,:,0]

    return X_train_new,Y_train,X_test_new,Y_test

In [5]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [6]:
def neural(channel,height,width,classes):
    input_shape = (channel,height,width)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (height,width,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    model.add(Flatten())
    model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(classes,activation="softmax",name="fc2"))
    return model


In [ ]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_out    = model.predict(sam_data, batch_size=32, verbose=1)
    yhat_label  = np.zeros(len(lab_data))
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind
        #
        tmp_vec = yhat_out[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        yhat_label[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN
    

In [ ]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]

nRun = 10
side_len = 22
num_class = 2
for dup_tag in range(2):
    for ran_tag in range(2):
        for fold in range(nRun):
            iFold = fold + 1
            X_train,Y_train,X_test,Y_test = DataSet(iFold,dup_tag,ran_tag)
            print('X_train shape : ',X_train.shape)
            print('Y_train shape : ',Y_train.shape)
            print('X_test shape : ',X_test.shape)
            print('Y_test shape : ',Y_test.shape)
            #print(X_train[0])
            #print(Y_train[0,1])

            # # model
            #model = neural(1,side_len,side_len,num_class)
            model = ResNet50(
                    weights=None,
                    classes=2)
            
            model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

            # # train
            model.fit(X_train, Y_train, epochs=10, batch_size=16)

            # # evaluate
            #model.evaluate(X_test, Y_test, batch_size=32)

            # #
            #models.append(model)


            # # save
            #save_path =("./my_resnet_model.h5_F%d") % (iFold)
            #model.save(save_path)

            #
            X_train,Y_train,X_test,Y_test = DataSet(iFold,0,ran_tag)
            train_N_TP,train_N_FP,train_N_TN,train_N_FN = get_statistics(model,X_train,Y_train)
            test_N_TP,test_N_FP,test_N_TN,test_N_FN = get_statistics(model,X_test,Y_test)
            all_TP_train.append(train_N_TP)
            all_FP_train.append(train_N_FP)
            all_TN_train.append(train_N_TN)
            all_FN_train.append(train_N_FN)
            all_TP_test.append(test_N_TP)
            all_FP_test.append(test_N_FP)
            all_TN_test.append(test_N_TN)
            all_FN_test.append(test_N_FN)


X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
Epoch 1/10
1410/1410 [==============================] - 2779s 2s/step - loss: 0.3709 - acc: 0.9177
Epoch 2/10
1410/1410 [==============================] - 2649s 2s/step - loss: 0.2839 - acc: 0.9305
Epoch 3/10
1410/1410 [==============================] - 1988s 1s/step - loss: 0.2651 - acc: 0.9291
Epoch 4/10
1410/1410 [==============================] - 1844s 1s/step - loss: 0.2620 - acc: 0.9298
Epoch 5/10
1410/1410 [==============================] - 1824s 1s/step - loss: 0.2276 - acc: 0.9305
Epoch 6/10
1410/1410 [==============================] - 1847s 1s/step - loss: 0.2129 - acc: 0.9362
Epoch 7/10
1410/1410 [==============================] - 1829s 1s/step - loss: 0.1895 - acc: 0.9418
Epoch 8/10
1410/1410 [==============================] - 1851s 1s/step - loss: 0.1869 - acc: 0.9376
Epoch 9/10
1410/1410 [==============================] - 1850s 1s/step - loss: 0.1781

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
Epoch 1/10
1410/1410 [==============================] - 1828s 1s/step - loss: 0.3603 - acc: 0.9177
Epoch 2/10
1410/1410 [==============================] - 1817s 1s/step - loss: 0.2686 - acc: 0.9291
Epoch 3/10
1410/1410 [==============================] - 1815s 1s/step - loss: 0.2336 - acc: 0.9340
Epoch 4/10
1410/1410 [==============================] - 1810s 1s/step - loss: 0.2305 - acc: 0.9312
Epoch 5/10
1410/1410 [==============================] - 1821s 1s/step - loss: 0.2629 - acc: 0.9262
Epoch 6/10
1410/1410 [==============================] - 1817s 1s/step - loss: 0.2203 - acc: 0.9340
Epoch 7/10
1410/1410 [==============================] - 1808s 1s/step - loss: 0.1969 - acc: 0.9326
Epoch 8/10
1410/1410 [==============================] - 1911s 1s/step - loss: 0.1872 - acc: 0.9362
Epoch 9/10
1410/1410 [==============================] - 1898s 1s/step - loss: 0.1555

In [ ]:
for dup_tag in range(2):
    for ran_tag in range(2):
        for iRun in range(nRun):
            print(all_TP_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FP_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_TN_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FN_train[dup_tag*2*nRun+ran_tag*nRun+iRun])
        for iRun in range(nRun):
            print(all_TP_test[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FP_test[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_TN_test[dup_tag*2*nRun+ran_tag*nRun+iRun])
            print(all_FN_test[dup_tag*2*nRun+ran_tag*nRun+iRun])


In [ ]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))
